## Analyze signals from each organoid, to determine viability and spike thresholds

In [1]:
import h5py
import numpy as np
import pandas as pd
import pickle as pkl
import os
import sys
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
from pprint import pprint
import glob

from oitools import ecr

In [2]:
root = "/cis/project/organoid"
os.listdir(root)

['April 19 2024',
 '2024May28 No window data ',
 'March 30 2024',
 'Nov_18_2024',
 '3_org_stim_vs_1_organoid',
 'Nov_18_2024_ecr_results3_no_window',
 'Dec_10_2024',
 'Dec_10_2024_ecr_results_no_window']

In [3]:
os.listdir(os.path.join(root, os.listdir(root)[6]))

['misc',
 'Overview Document Run 28 Stimulation Experiment 3 Organoids vs 1 Organoid.docx',
 '241018']

In [4]:
import os

def get_folder_size_in_mb(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            total_size += os.path.getsize(file_path)
    size_mb = total_size / (1024 * 1024*1024)
    return size_mb

root = "/cis/project/organoid"

# List all items in the root directory
items = os.listdir(root)

# Iterate over the items and check if they are directories
for item in items:
    item_path = os.path.join(root, item)
    if os.path.isdir(item_path):
        folder_size_mb = get_folder_size_in_mb(item_path)
        print(f"Directory: {item}, Size: {folder_size_mb:.2f} GB")

Directory: April 19 2024, Size: 12.03 GB
Directory: 2024May28 No window data , Size: 13.31 GB
Directory: March 30 2024, Size: 18.59 GB
Directory: Nov_18_2024, Size: 10.69 GB
Directory: 3_org_stim_vs_1_organoid, Size: 10.38 GB
Directory: Nov_18_2024_ecr_results3_no_window, Size: 0.08 GB
Directory: Dec_10_2024, Size: 58.30 GB
Directory: Dec_10_2024_ecr_results_no_window, Size: 1.07 GB


In [5]:
os.listdir("/cis/project/organoid/Dec_10_2024_ecr_results_no_window/241018/M07915/Stimulation/000295")

['data.raw_20241213_18h15m.pkl']

In [6]:
# Path of source .h5 files
root = "/cis/project/organoid"
path_source_files = os.path.join(root, os.listdir(root)[3])
print(path_source_files)

# Path where results will be stored
path_results = '/cis/project/organoid/Dec_10_2024_ecr_results_no_window'

/cis/project/organoid/Nov_18_2024


### Get a list of files that we'll analyze

In [7]:
# Get a list of files that we'll analyze

# Get list of all h5 files in the target directory
filenames = os.listdir(path_source_files)
filenames = [fn for fn in filenames if fn.endswith('.h5')]

# Get list of file sizes
sizes = []
for f in filenames:
    sizes.append(os.stat(os.path.join(path_source_files, f)).st_size)

# Excluding files that are relatively small
thresh = np.median(sizes)/4
print(f'Exluding files of size less than {thresh/1e6:0.3f} MB.')
filenames = [fn for fn, sz in zip(filenames, sizes) if sz>=thresh]

print(f'\nAssessing these h5 files in {path_source_files}.')
pprint(filenames)

Exluding files of size less than 2764.922 MB.

Assessing these h5 files in /cis/project/organoid/Nov_18_2024.
['data.raw.h5']


### Print out well names for each h5 file

In [8]:
# Print out well names for each file

for filename in filenames:
    fullname = os.path.join(path_source_files, filename)
    ephys_file = h5py.File(fullname, 'r')
    
    print(f'\n{filename}:\n\t', end='')
    print(list(ephys_file['recordings']['rec0000'].keys()))


data.raw.h5:
	['well000', 'well001', 'well002', 'well003', 'well004', 'well005']


## Analyze data well by well (organoid by organoid)

In [ ]:
fs = 10000  # Sampling rate for all Maxwell spike data

## Values used to filter out invidual spikes, and channel with low spike count
thresh_spike_amp_percentile = 5  # Discard the low P percentile of spikes, which may just be noise
thresh_spikes_per_chan_std = 2  # Discard channels with < mean(n_spikes)+spike_cnt_thresh_std*std(n_spikes), on log scale

## Values used for Puppa's ECR algorithm
amp_thresh_percentile = None
amp_thresh_std = 1
adj_threshold = 1.0
time_resolution = 1/fs
# Values used in Puppa paper, on real spike data...
raster_dur = 0.0005
corr_type = 'cc'  ## 'cc'==cross-correlation, 'pearson' not yet implemented
epsilon = 0.003
T_list = np.array([0.020, 0.0175, 0.016])
sigma_list = np.array([0.0004, 0.00055, 0.0007])

parameters = {}
parameters['epsilon'] = epsilon
parameters['T_list'] = T_list
parameters['sigma_list'] = sigma_list
parameters['amp_thresh_percentile'] = amp_thresh_percentile
parameters['amp_thresh_std'] = amp_thresh_std
parameters['raster_dur'] = raster_dur
parameters['corr_type'] = corr_type
parameters['adj_threshold'] = adj_threshold
parameters['time_resolution'] = time_resolution
ss_ecr = ecr.SuperSelective(epsilon=epsilon,
                                 T_list=T_list,
                                 sigma_list=sigma_list,
                                 amp_thresh_percentile=amp_thresh_percentile,
                                 amp_thresh_std=amp_thresh_std,
                                 n_corr_peaks_max=4,
                                 raster_dur=raster_dur,
                                 corr_type=corr_type,
                                 adj_threshold=adj_threshold,
                                 time_resolution=time_resolution,
                                 use_multiprocessing=True,
                                 verbose=False)

os.makedirs(path_results, exist_ok=True)

for i_file, filename in enumerate(filenames):
    print(f'Processing file {i_file+1} of {len(filenames)}, {filename}...')
    
    data_to_save = {}
    data_to_save['filename'] = filename
    data_to_save['parameters'] = parameters
    
    fullname = os.path.join(path_source_files, filename)
    ephys_file = h5py.File(fullname, 'r')
    well_keys = list(ephys_file['recordings']['rec0000'].keys())

    for i_well, well in enumerate(well_keys):
        t_process_start = time.time()
        print(f'Processing well {i_well+1} of {len(well_keys)}, {well}...')
        df_spikes = pd.DataFrame(np.array(ephys_file['recordings']['rec0000'][well]['spikes']))
        
        # Determine spike amplitude threshold
        thresh_spike_amp = np.percentile(df_spikes['amplitude'], thresh_spike_amp_percentile)
        print(f'{thresh_spike_amp_percentile}% spike amplitude percentile is {thresh_spike_amp}.')       

        # Filter spikes by amplitude
        df_spikes = df_spikes[df_spikes['amplitude'] > thresh_spike_amp]
        
        # Get spike times
        df_spikes['time_sec'] = df_spikes['frameno'] / fs
        t_sig_start = np.min(df_spikes['time_sec'])
        df_spikes['time_sec'] = df_spikes['time_sec'] - t_sig_start
        t_sig_last = np.max(df_spikes['time_sec'])
        print(f'Recording duration is {t_sig_last:0.4f} seconds.')
        
        # Aggregate spikes into separate channels
        chan_nums = np.unique(df_spikes['channel'])
        t_spikes_in_chan = []
        n_spikes_in_chan = np.zeros(len(chan_nums))
        for i_chan, chan in enumerate(chan_nums):
            t_spikes_in_chan.append(df_spikes[df_spikes['channel']==chan]['time_sec'].values)
            n_spikes_in_chan[i_chan] = len(t_spikes_in_chan[-1])

        # Detemine threshold for spike per channel
        thresh_spikes_per_chan = np.mean(np.log(n_spikes_in_chan)) + \
                                 thresh_spikes_per_chan_std * np.std(np.log(n_spikes_in_chan))
        print(f'Channel spike count threshold is {thresh_spikes_per_chan}')

        ### We'll keep all channels and filter out low-amplitude correlation peaks
        ### in the ECR computation, rather than filter out individual channels here.
#         # Filter out channels with too few spikes
#         ix_keep = np.argwhere(n_spikes_in_chan >= thresh_spikes_per_chan)[:, 0]
#         t_spikes_in_chan = [t for n, t in zip(n_spikes_in_chan, t_spikes_in_chan) if n >= thresh_spikes_per_chan]
#         n_spikes_in_chan = n_spikes_in_chan[ix_keep]
#         chan_nums = chan_nums[ix_keep]
        print(f'Number of channels kept is {len(n_spikes_in_chan)}.')

        
#         ## Plot some things, if stepping through this loop with pdb debugger
#         plt.figure(figsize=(10, 10))
        
#         # Spike counts per channel, sorted
#         plt.subplot(2, 2, 1)
#         plt.semilogy(np.sort(n_spikes_in_chan), '.')
#         plt.xlabel('Channel rank order')
#         plt.ylabel('Spike count')
#         plt.title('Spike counts per by channel')
#         plt.grid()
        
#         # Histogram of all spike times
#         plt.subplot(2, 2, 2)
#         plt.hist(df_spikes['time_sec'], bins=100)
#         plt.grid()
#         plt.xlabel('Spike time')
#         plt.ylabel('Counts')
#         plt.title('Spike time histogram')

#         # Histogram of all spike amlitudes
#         plt.subplot(2, 2, 3)
#         plt.hist(df_spikes['amplitude'], bins=100)
#         plt.grid()
#         plt.xlabel('Spike amplitude')
#         plt.ylabel('Counts')
#         plt.title('Spike amplitude histogram')

#         plt.pause(0.1)
#         plt.draw()
#         import pdb
#         pdb.set_trace()


        # Estimate Effective Connectivity Reconstruction
        adj_matrix_predicted, votes, corr_peaks = ss_ecr(t_spikes_in_chan)


        # Put results in a dict and save as a pickle file
        data_to_save['results'] = {}
        data_to_save['results']['well'] = well
        data_to_save['results']['thresh_spike_amp'] = thresh_spike_amp
        data_to_save['results']['thresh_spikes_per_chan'] = thresh_spikes_per_chan
        data_to_save['results']['channel_numbers'] = chan_nums
        data_to_save['results']['adj_matrix_predicted'] = adj_matrix_predicted
        data_to_save['results']['votes'] = votes
        data_to_save['results']['corr_peaks'] = corr_peaks

        with open(os.path.join(path_results, filename[:-2]+'pkl'), 'wb') as f:
            pkl.dump(data_to_save, f, protocol=pkl.HIGHEST_PROTOCOL)
        
        print(f'Done. Processing took {time.time()-t_process_start:0.2f} seconds.\n\n')

Processing file 1 of 1, data.raw.h5...
Processing well 1 of 6, well000...
5% spike amplitude percentile is -22.629671096801758.
Recording duration is 600.0772 seconds.
Channel spike count threshold is 5.949471816287769
Number of channels kept is 685.

Using maximum of 15 CPUs for processing.

Done. Processing took 53.76 seconds.


Processing well 2 of 6, well001...
5% spike amplitude percentile is -15.84598445892334.
Recording duration is 600.0689 seconds.
Channel spike count threshold is 5.842258973142006
Number of channels kept is 777.

Using maximum of 15 CPUs for processing.

Done. Processing took 63.08 seconds.


Processing well 3 of 6, well002...
5% spike amplitude percentile is -20.35711097717285.
Recording duration is 600.0415 seconds.
Channel spike count threshold is 6.512066747803693
Number of channels kept is 1006.

Using maximum of 15 CPUs for processing.

Done. Processing took 368.01 seconds.


Processing well 4 of 6, well003...
5% spike amplitude percentile is -13.0329799